In [1]:
#include <iostream>
#include <xtensor/xarray.hpp>
#include <xtensor/xio.hpp>
#include <xtensor/xview.hpp>
#include <xtensor/xarray.hpp>
#include <xtensor/xfixed.hpp>
#include <xtensor/xio.hpp>
#include <xtensor/xtensor.hpp>
#include <xtensor/xadapt.hpp>
#include <xtensor/xarray.hpp>
#include <xtensor/xaxis_slice_iterator.hpp>
#include <xtensor/xaxis_iterator.hpp>
#include <xtensor/xio.hpp>
#include <xtensor/xarray.hpp>
#include <xtensor/xindex_view.hpp>
#include <xtensor/xchunked_array.hpp>
#include <xtensor/xfixed.hpp>
#include <cstddef>
#include <vector>
#include <xtensor/xarray.hpp>
#include <xtensor/xadapt.hpp>
#include <istream>
#include <fstream>
#include <iostream>
#include <xtensor/xarray.hpp>
#include <xtensor/xcsv.hpp>
#include <iostream>
#include <fstream>
#include <functional>
#include <cstdio>
#include <xtensor/xcsv.hpp>
#include <xcpp/xdisplay.hpp>
#include <xtensor/xsort.hpp>

using namespace xt;
using namespace xt::placeholders;  // to enable _ syntax

#include <iostream>
#include <fstream>
using namespace std;

# DBSCANRN

# Load data

In [2]:
ifstream input_file;
input_file.open ("lecture_dataset.txt");
auto data = load_csv<double>(input_file);
input_file.close();

data.reshape(data.shape());
xarray<double> X = view(data, all(), range(0, 2));

# Similarity



In [3]:
float cosine_dissimilarity(const xarray<double>& a, const xarray<double>& b){ 
    auto numerator = sum(a * b);
    auto denominator = sqrt(sum(pow(a, 2))) * sqrt(sum(pow(b, 2)));
    auto result = numerator / denominator;
    return 1 - result(0);
}

In [4]:
auto a = row(X, 0); 
auto b = row(X, 1);
    
cosine_dissimilarity(a,b)

0.0156116f

# Clusterization

In [9]:
auto x = empty<double>({1})

In [10]:
auto all_point_indices = arange(10);
xarray<int> idx = from_indices(argwhere(not_equal(all_point_indices, 4)));
view(idx, range(1,all()))

1
2
3
5
6
7
8
9


In [8]:
auto all_point_indices = arange(5);
// auto s = all_point_indices.shape();
auto x = col(from_indices(argwhere(not_equal(all_point_indices, 2))),0);
auto s = x.shape();
std::cout << adapt(s) << std::endl

{4}


@0x7fabe14b3ba0

In [3]:
xarray<int> a = {2, 1, 7, 4, 454};
auto ax = view(a, range(1, all()));

xarray<int> idx = argsort(ax);

xarray<float> c = {244, 41, 47, 6565, 5};
xarray<int> cx = view(c, range(1, all()));

index_view(cx, idx);
// concatenate(xtuple(a, b))
c.reshape(5);
auto s = c.shape();
std::cout << adapt(s) << std::endl;

input_line_9:2:2: error: no template named 'xarray'; did you mean 'std::array'?
 xarray<int> a = {2, 1, 7, 4, 454};
 ^~~~~~
 std::array
/root/anaconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/array:94:12: note: 'std::array' declared here
    struct array
           ^
input_line_9:2:2: error: too few template arguments for class template 'array'
 xarray<int> a = {2, 1, 7, 4, 454};
 ^
/root/anaconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/array:94:12: note: template is declared here
    struct array
           ^
input_line_9:3:28: error: use of undeclared identifier 'all'
auto ax = view(a, range(1, all()));
                           ^
input_line_9:4:1: error: no template named 'xarray'; did you mean 'std::array'?
xarray<int> idx = argsort(ax);
^~~~~~
std::array
/root/anaconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/a

Interpreter Error: 

In [4]:
auto list_contains(list<int> my_list, int element){
    return (find(my_list.begin(), my_list.end(), element) != my_list.end());
}

In [6]:
auto dbscanrn(const xarray<float>& X, int k)
{
    // each data point can be in one of 3 stages
    int NOT_VISITED = -1; // not visited point
    int VISITED = 0; // non-core point
    int CLUSTERED = 1; // core point

    // initial setup
    int n = X.shape()[0];
    xarray<double> cluster = ones<double>({n}) * -1; // cluster register
    xarray<double> state = ones<double>({n}) * NOT_VISITED; // state register
    int cluster_id = 1;
    auto all_point_indices = arange(n);
    map<int, vector<int>> point_rnn;
    map<int, xarray<int>> point_knn;
    
    auto get_knn = [&] (int current_index){
        // remove current index from neighbors
        xarray<int> neighbor_indices = col(
            from_indices(argwhere(not_equal(all_point_indices, current_index))), 0
        );
        xarray<float> neighbor_similaritys = {0};
        for (auto& neighbor_index : neighbor_indices) {
            auto similarity = cosine_dissimilarity(
                row(X, neighbor_index), row(X, current_index)
            );
            neighbor_similaritys = concatenate(
                xtuple(neighbor_similaritys, xarray<float> {similarity})
            );
        };
        xarray<int> sort_indices = argsort(view(neighbor_similaritys, range(1, all())));
        xarray<int> neighbor_indices_sorted = index_view(neighbor_indices, sort_indices); 
        point_knn[(int) current_index] = view(neighbor_indices_sorted, range(0, k));  
    };    
    
    auto get_pointwise_rnn = [&] (int current_index){
        vector<int> rnn;
        for (auto& neighbor_index : point_knn[(int) current_index]) {
            if ((point_knn[(int) neighbor_index], (int) current_index)) {
                rnn.push_back((int) neighbor_index);
            };
        };
        point_rnn[(int) current_index] = rnn;
        // return rnn;
    };
    
    auto get_rnn = [&] (){
        for (int current_index = 0; current_index < n; current_index++) {
            get_knn((int) current_index);
        };
        for (int current_index =  0; current_index < n; current_index++) {
            get_pointwise_rnn((int) current_index);
        };
    };
    
    get_rnn();
    xcpp::display(point_rnn);

//     // extend cluster
//     function < void( int ) > search = [&] (int current_index) -> void {
//         list<int> neighbor_indices = get_neighbors(current_index);
//         if ((int) neighbor_indices.size() >= minPts){
//             state(current_index) = CLUSTERED;
//             cluster(current_index) = cluster_id;
//             for (auto& neighbor_index : neighbor_indices) {
//                 if (state(neighbor_index) == VISITED || state(neighbor_index) == NOT_VISITED){
//                     state(neighbor_index) = CLUSTERED;
//                     cluster(neighbor_index) = cluster_id;
//                     search(neighbor_index);
//                 };
//             };
//         } else {
//             state(current_index) = VISITED;
//         };
//     };

//     while (any(equal(state, NOT_VISITED))){
//         xarray<int> idx = from_indices(argwhere(equal(state, NOT_VISITED)));
//         search(idx(0,0));
//         cluster_id++;
//     }
    return cluster;
}

auto y_pred = dbscanrn(X, 4);
// y_pred

{ 0 => {}, 1 => { 7, 0, 2, 11 }, 2 => { 11, 0, 7, 1 }, 3 => { 9, 4, 8, 10 }, 4 => { 3, 8, 9, 10 }, 5 => { 11, 2, 6, 0 }, 6 => { 5, 11, 2, 0 }, 7 => { 0, 1, 2, 11 }, 8 => { 4, 3, 9, 10 }, 9 => { 3, 4, 8, 10 }, 10 => { 9, 3, 4, 8 }, 11 => { 2, 0, 5, 7 } }

In [22]:
xarray<int> sort_indices = {6, 1, 0, 10, 4, 7, 3, 2, 8, 9, 5};
xarray<int> neighbor_indices = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11};
index_view(neighbor_indices, sort_indices)

{ 7, 2, 1, 11, 5, 8, 4, 3, 9, 10, 6 }

# Save output to csv

In [10]:
ofstream out_file;
out_file.open ("example.csv");
dump_csv(out_file, view(y_pred, all(), newaxis()));
out_file.close();